# RPKI statistics

https://lirportal.ripe.net/certification/content/static/statistics/ripencc.tal.txt

Also available from NRO adoption stats,
https://ftp.ripe.net/pub/stats/ripencc/nro-adoption/2023/07/01/rir-adoption.txt

In [1]:
import netaddr
import requests
import bz2

from io import StringIO

import pandas as pd

In [4]:
def ipv4_prefix_from_row(row: pd.DataFrame) -> netaddr.IPRange:
    start = netaddr.IPAddress(row.prefix)
    return netaddr.IPRange(start, start + row.prefix_size)

def ipv6_prefix_from_row(row: pd.DataFrame) -> netaddr.IPNetwork:
    return f"{row.prefix}/{row.prefix_size}"


def rpki_cover_stats(date_str: str, roas_csv_url: str, delegated_stats_url: str):
    df_roas = pd.read_csv(roas_csv_url)

    prefixes = df_roas['IP Prefix']

    roa_space_v4 = netaddr.IPSet(prefixes[prefixes.str.contains("\.")])
    roa_space_v6 = netaddr.IPSet(prefixes[prefixes.str.contains(":")])

    assert prefixes[prefixes.str.contains(":")].size + prefixes[prefixes.str.contains("\.")].size == prefixes.size

    raw_delegated_extended = requests.get(delegated_stats_url).content
    raw_delegated_extended = StringIO(bz2.decompress(raw_delegated_extended).decode('ascii'))

    raw_delegated_extended.seek(0)
    df_delegated_extended = pd.read_csv(
        raw_delegated_extended,
        sep="|",
        skiprows=4,
        names=['rir', 'country', 'afi', 'prefix', 'prefix_size', 'date', 'status', 'uuid'],
        dtype={'rir': 'category', 'country': 'category',  'afi': 'category', 'prefix': str, 'size': int, 'date': str, 'status': 'category', 'uuid': str})

    df_delegated_extended.date = pd.to_datetime(df_delegated_extended.date, format='%Y%m%d')
    df_ripe_member_space = df_delegated_extended[(df_delegated_extended.status != 'available') & (df_delegated_extended.status != 'reserved')]


    df_ripe_member_space = df_delegated_extended[(df_delegated_extended.status != 'available') & (df_delegated_extended.status != 'reserved')]


    ipv4 = df_ripe_member_space[df_ripe_member_space.afi == 'ipv4'].apply(ipv4_prefix_from_row, axis=1, result_type='reduce')
    ipv6 = [netaddr.IPNetwork(p) for p in df_ripe_member_space[df_ripe_member_space.afi == 'ipv6'].apply(ipv6_prefix_from_row, axis=1)]

    ripe_space_v4 = netaddr.IPSet(ipv4)
    ripe_space_v6 = netaddr.IPSet(ipv6)
    
    print(date_str)
    print("ipv4 covered: {:.2%}".format((ripe_space_v4 & roa_space_v4).size/ripe_space_v4.size))
    print("ipv6 covered: {:.2%}".format((ripe_space_v6 & roa_space_v6).size/ripe_space_v6.size))
    
    return df_ripe_member_space

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:17: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:17: SyntaxWarning: invalid escape sequence '\.'
/var/folders/m4/qtvb8ff50ng0yw7xyt7t0n9c0000gp/T/ipykernel_38391/2961649739.py:14: SyntaxWarning: invalid escape sequence '\.'
  roa_space_v4 = netaddr.IPSet(prefixes[prefixes.str.contains("\.")])
/var/folders/m4/qtvb8ff50ng0yw7xyt7t0n9c0000gp/T/ipykernel_38391/2961649739.py:17: SyntaxWarning: invalid escape sequence '\.'
  assert prefixes[prefixes.str.contains(":")].size + prefixes[prefixes.str.contains("\.")].size == prefixes.size


In [5]:
rpki_cover_stats(
    "2023-07-01",
        "https://ftp.ripe.net/rpki/ripencc.tal/2023/07/01/roas.csv.xz",
    "https://ftp.ripe.net/pub/stats/ripencc/2023/delegated-ripencc-extended-20230701.bz2"
)

2023-07-01
ipv4 covered: 62.44%
ipv6 covered: 37.19%


,rir,country,afi,prefix,prefix_size,date,status,uuid
0,ripencc,PS,ipv4,1.178.112.0,4096,2007-11-26,allocated,08a0e97a-1d16-423d-9ed2-95c80e18d882
1,ripencc,PS,ipv4,1.178.128.0,4096,2007-11-26,allocated,08a0e97a-1d16-423d-9ed2-95c80e18d882
2,ripencc,PS,ipv4,1.178.208.0,4096,2010-06-25,allocated,08a0e97a-1d16-423d-9ed2-95c80e18d882
3,ripencc,ES,ipv4,1.178.224.0,8192,2010-06-25,allocated,7bdefdac-8071-46bf-9b36-e39850aca684
4,ripencc,PS,ipv4,1.179.40.0,2048,2009-05-18,allocated,08a0e97a-1d16-423d-9ed2-95c80e18d882
...,...,...,...,...,...,...,...,...
160161,ripencc,AM,ipv6,2001:7f9:8::,48,2016-06-30,assigned,37e73a9f-bdfc-42f4-bec0-71f89d498241
160162,ripencc,PL,ipv6,2001:7f9:c::,48,2017-01-31,assigned,e95fa156-2556-49c3-a9e6-f1cd669e776e
160163,ripencc,NL,ipv6,2001:7fb::,32,2007-06-05,assigned,db3b7f71-2600-4112-84bc-5ba6f642e97a
160164,ripencc,NL,ipv6,2001:7fd::,32,2003-08-29,assigned,db3b7f71-2600-4112-84bc-5ba6f642e97a


In [ ]:
rpki_cover_stats(
    "2022-07-01",
    "https://ftp.ripe.net/rpki/ripencc.tal/2022/07/01/roas.csv.xz",
    "https://ftp.ripe.net/pub/stats/ripencc/2022/delegated-ripencc-extended-20220701.bz2"
)

In [ ]:
rpki_cover_stats(
    "2021-07-01",
    "https://ftp.ripe.net/rpki/ripencc.tal/2021/07/01/roas.csv.xz",
    "https://ftp.ripe.net/pub/stats/ripencc/2021/delegated-ripencc-extended-20210701.bz2"
)

In [ ]:
rpki_cover_stats(
    "2020-07-01",
    "https://ftp.ripe.net/rpki/ripencc.tal/2020/07/01/roas.csv.xz",
    "https://ftp.ripe.net/pub/stats/ripencc/2020/delegated-ripencc-extended-20200701.bz2"
)

In [ ]:
df_ripe_member_space = rpki_cover_stats(
    "2023-07-01",
        "https://ftp.ripe.net/rpki/ripencc.tal/2023/07/01/roas.csv.xz",
    "https://ftp.ripe.net/pub/stats/ripencc/2023/delegated-ripencc-extended-20230701.bz2"
)

In [ ]:
df_ripe_member_space